![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### MySQL and MariaDB for Python Developers

# Adding calculated values to query results

In this project, you will enhance MySQL to perform operations not available by default in queries.

You will need access to a MySQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See ` https://hub.docker.com/_/mysql` for details on that option.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Calculating geometric mean**

In this task, we will continue to use the airline tweets table that has been used in previous projects.  A solution is available in the file `make_psql.py` in this projects.  Sometimes we with to perform queries that describe aggregations of the data.  For example:

In [3]:
import pandas as pd
import mysql.connector
cred = dict(user='ine_student', password='ine-password', database='ine')
conn = mysql.connector.connect(**cred)
cur = conn.cursor()

In [5]:
sql = """
SELECT airline, avg(airline_sentiment_confidence), avg(negativereason_confidence)
FROM Tweets
GROUP BY airline;
"""
cur.execute(sql)
pd.DataFrame(cur.fetchall(), columns=['airline', 'avg_sentiment_conf', 'avg_neg_conf'])

,airline,avg_sentiment_conf,avg_neg_conf
0,Delta,0.847048,0.573097
1,United,0.896252,0.627285
2,Southwest,0.867695,0.601508
3,US Airways,0.919169,0.687172
4,Virgin America,0.838760,0.556985
5,American,0.912919,0.665752


For this task, you would like to make a similar report, but using geometric mean rather than arithmetic mean.   Creating a VIEW that contains both mean and geometric mean as columns is a good approach that can be queried whenever the underlying data updates.  For extra credit, you could add the midrange.

**Solution**

In [15]:
sql_conf = """
CREATE OR REPLACE VIEW airline_sentiment (
    airline, mean, gmean, midrange) AS 
SELECT airline, 
       avg(airline_sentiment_confidence),
       exp(sum(log(airline_sentiment_confidence)) / count(airline_sentiment_confidence)),
       (max(airline_sentiment_confidence) + min(airline_sentiment_confidence)) / 2
FROM Tweets
GROUP BY airline;
"""
cur.execute(sql_conf)

In [17]:
cur.execute("SELECT * FROM airline_sentiment")
pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

,airline,mean,gmean,midrange
0,Delta,0.847048,0.820950,0.66815
1,United,0.896252,0.876755,0.66750
2,Southwest,0.867695,0.844234,0.66765
3,US Airways,0.919169,0.903026,0.67000
4,Virgin America,0.838760,0.813192,0.67410
5,American,0.912919,0.896896,0.66835


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Calculating median**

Unfortunately, the aggregation `MEDIAN()` is not one provided by MySQL.  See if you can find it yourself, per airline, using SQL rather than Python as much as possible.

Hint: It is *possible* in pure MySQL, but **much** easier if you allow yourself part of the logic in Python.

**Solution**

In [25]:
results = {}
cur.execute("SELECT DISTINCT airline FROM Tweets")
for row in cur.fetchall():
    airline = row[0]
    newcur = conn.cursor()
    newcur.execute(f"SELECT count(*) FROM Tweets WHERE airline='{airline}'")
    mid = newcur.fetchone()[0]//2
    sql = (f"SELECT airline_sentiment_confidence "
           f"FROM Tweets LIMIT 1 OFFSET {mid}")
    newcur.execute(sql)
    results[airline] = newcur.fetchone()[0]
    
results

{'Delta': 0.6556,
 'United': 0.6871,
 'Southwest': 1.0,
 'US Airways': 0.6735,
 'Virgin America': 1.0,
 'American': 0.3356}

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)